In [1]:
%load_ext autoreload
%autoreload 2

import radon
import torch

_avail_gpus = torch.cuda.device_count()
GPU_ID = 2  # Default GPU ID
if _avail_gpus == 0:
    print("No GPUs available -- will need one for cuda")
    device = "cpu"
else:
    print(f"Available GPUs: {_avail_gpus}")
    print(f"Using GPU ID {GPU_ID} : {torch.cuda.get_device_name(GPU_ID)}")
    torch.cuda.set_device(GPU_ID)
    device = f"cuda:{GPU_ID}"

CUDA backend loaded successfully.
Available GPUs: 4
Using GPU ID 2 : NVIDIA L40S


In [2]:
a = torch.rand(3, device=GPU_ID)
b = torch.rand(3, device=GPU_ID)
c = torch.zeros_like(a, device=GPU_ID)
radon.add_arrays(a, b, c)
print("Result:", c)

Result:Hello from CUDA thread 0: 0.01 + 0.63 = 0.64
Hello from CUDA thread 1: 0.46 + 0.17 = 0.63
Hello from CUDA thread 2: 0.69 + 0.63 = 1.32
 tensor([0.6408, 0.6274, 1.3195], device='cuda:2')


In [3]:
r = radon.Radon()
print("Radon object created:", r)
print("Radon version:", r.version)

Radon object created: <radon.radon.Radon object at 0x746d476cf3b0>
Radon version: 0.1.0
